# Define Functions and Run Initial Query

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import json

pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_columns = 500
pd.options.display.max_rows = 20

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    TestGroup = np.array(('364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275')).astype(int)
    df['TestGroup'] = (df['Employee'].isin(TestGroup)).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    df['Proactive Launched']=(df['Date']>=datetime(2023,11,10)).astype(int)
    df['Constant'] =1
    return df

In [ ]:
dfp = run_query(
"""
SELECT
    element_at(VHE.edp_raw_data_map, 'EventName'),
    element_at(VHE.edp_raw_data_map, 'type'),
    COUNT(*)
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, 'Scope') = 'ExWo'
    AND DATE(edp_updated_date) > DATE('2023-11-10')
GROUP BY
    element_at(VHE.edp_raw_data_map, 'EventName'),
    element_at(VHE.edp_raw_data_map, 'type')
""")

In [2]:
vhe_dfp = run_query(
"""
select *
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND element_at(VHE.edp_raw_data_map, 'Scope')='GenerativeAISearchBotVoice'
    AND element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') in ('That worked!', 'Still need help', 'Wrong issue')
    AND (CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) in (573190, 573276, 572909, 573192, 573585, 573573, 572815, 552121, 576911, 572247)
    OR CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) is null)
    AND DATE(edp_updated_date) > DATE('2023-11-10')
""")

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_17932\494237659.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


In [3]:
df = vhe_dfp.copy()
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))
display(vhe_dfp.head(),json_df.head(),json_df.columns)
display(pd.DataFrame(json_df['ExtraData_inputMessage'].value_counts()).sort_values('ExtraData_inputMessage',ascending=False).head(20))
json_df.to_excel('JsonData.xlsx')
del df, json_df


,batchid,edp_raw_data,edp_headers,meta_kafka_metadata,eventid,op,edp_updated_date,dms_time_stamp,edp_raw_data_map
0,58362,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",3,309057987...",524560498,I,2023-11-20,1700495836867,"{""ExtraData_totalTime"":""1.762920686043799"",""_h..."
1,58030,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",2,308453446...",1964753309,I,2023-11-14,1699982154634,"{""ExtraData_totalTime"":""1.5159555840073153"",""_..."
2,58585,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",3,309436650...",-1563379467,I,2023-11-24,1700843880498,"{""ExtraData_totalTime"":""1.4487825069809332"",""_..."
3,58360,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",3,309053683...",-93814717,I,2023-11-20,1700492259973,"{""ExtraData_totalTime"":""2.121073530986905"",""_h..."
4,57807,"{""Name"":""RequestSummaryVoice"",""Scope"":""Generat...","{""/encryption/AES_KMS/encryptedKey"":""\u0001\u0...","[""platform.ingest.app.home-events"",1,308128762...",222118955,I,2023-11-11,1699667050680,"{""ExtraData_totalTime"":""3.0828951460134704"",""_..."


,ExtraData_totalTime,_header_client,Identities_callSid,MetaData_RequestContext_Host,MetaData_RequestContext_User-Agent,ExtraData_openAITime,Name,_header_eventContext_producer,ExtraData_isLive,_header_category,_header_eventId,MetaData_RequestContext_Accept,_header_domain,_header_schema,EventId,_header_object,ExtraData_agentId,_header_eventContext_category,MetaData_RequestContext_Accept-Encoding,ExtraData_botResponse,_header_eventContext_sourceType,ExtraData_inputMessage,ExtraData_helixTime,_header_eventContext_region,_header_eventContext_ingestionTimestamp,ExtraData_intentTime,_header_eventContext_endpoint,ExtraData_promptTranscript,_header_source,MetaData_RequestContext_X-Forwarded-For,ExtraData_ozmoTime,ExtraData_Time,MetaData_ServerTime,MetaData_Client,MetaData_RequestContext_X-Forwarded-Proto,ExtraData_endRequestTime,Time,_header_tags,_header_timestamp,ExtraData_truncatePromptTime,ExtraData_transcriptsTime,MetaData__eventType,MetaData_RequestContext_X-Amzn-Trace-Id,MetaData_ProcessorType,ExtraData_messageType,ExtraData_version,ExtraData_totalTokens,MetaData_IsTest,Identities_messageSid,ExtraData_embeddingsTime,ExtraData_startRequestTime,ExtraData_rerankerTime,ExtraData_helixNewTime,_header_action,MetaData_RequestContext_Content-Type,Scope,FullName,MetaData_RequestContext_X-Forwarded-Port,_header_eventContext_correlationId,Identities_SessionId,ExtraData_deviceDetectionTime
0,1.762920686043799,verizon,CA22b8a8d6e00e1677927d49a4cf68ba63,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,0.5554858900140971,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,25b6de16-ca12-471e-812a-c0d57389ab11,*/*,event,analytics.ingestion.v1,25b6de16-ca12-471e-812a-c0d57389ab11,GenerativeAISearchBotVoice,572247,analytics,"gzip, deflate",I apologize for the confusion. Can you please ...,HTTP,Wrong issue,0,us-east-2,1700494317332,0,ingestion,"[{""callSid"":""CA22b8a8d6e00e1677927d49a4cf68ba6...",EDaaS|Eventing|US,34.205.188.78,0,2023-11-20T15:31:57.332Z,2023-11-20T15:32:35.530201873Z,verizon,https,2023-11-20 09:31:57.277404,2023-11-20T15:31:57.332Z,[],1700494355530,0.014942603011149913,0,event,Root=1-655b7bed-1acbc00c2d7868803f77cb68,EIP,Search,1.0.7,2181,false,e734d69d-e16c-497a-b2a8-d77d173ac5b9,0.08500376698793843,2023-11-20 09:31:55.501893,0.7112420019693673,0.07900580600835383,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,27f05ed5-1315-4ca1-a7cf-6652f160af63,e9cd6e9d-bb96-4983-a9df-ddd05ca899b6,0.0002744199591688812
1,1.5159555840073153,verizon,CAd9a41283addc093fade7772abff42faf,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,0.38727145199663937,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,f1c8362f-04cc-4ccc-90ca-88694e6f160e,*/*,event,analytics.ingestion.v1,f1c8362f-04cc-4ccc-90ca-88694e6f160e,GenerativeAISearchBotVoice,572909,analytics,"gzip, deflate",I apologize for the confusion. Can you please ...,HTTP,Wrong issue,0,us-east-2,1699980501788,0,ingestion,"[{""callSid"":""CAd9a41283addc093fade7772abff42fa...",EDaaS|Eventing|US,34.205.188.78,0,2023-11-14T16:48:21.788Z,2023-11-14T16:48:21.801886242Z,verizon,https,2023-11-14 10:48:21.732033,2023-11-14T16:48:21.788Z,[],1699980501801,0.019133236026391387,0,event,Root=1-6553a4d5-0590fcbe444c7dd121542eb6,EIP,Search,1.0.7,2181,false,93a56888-f0ad-426a-a659-79b62e9ffcae,0.06736093194922432,2023-11-14 10:48:20.200672,0.6729111470049247,0.09413429297273979,created,application/json,GenerativeAISearchBotVoice,GenerativeAISearchBotVoice_RequestSummaryVoice,443,8ac97f22-f1f4-4984-8d20-ed6e388d4052,01f574f4-787b-4e99-99cb-87edda3ad6ee,0.00031441199826076627
2,1.4487825069809332,verizon,CA4abe07337a4161bc5d34bee5fb425775,ingestion.events.asurion.com,Python/3.10 aiohttp/3.8.5,0.3199149579741061,RequestSummaryVoice,eip-ingestion-data-science,true,Analytics,060d594b-c8a3-453a-add2-3496bc33d285,*/*,event,analytics.ingestion.v1,060d594b-c8a3-453a-add2-3496bc33d285,GenerativeAISearchBotVoice,572247,analytics,"gzip, def

Index(['ExtraData_totalTime', '_header_client', 'Identities_callSid',
       'MetaData_RequestContext_Host', 'MetaData_RequestContext_User-Agent',
       'ExtraData_openAITime', 'Name', '_header_eventContext_producer',
       'ExtraData_isLive', '_header_category', '_header_eventId',
       'MetaData_RequestContext_Accept', '_header_domain', '_header_schema',
       'EventId', '_header_object', 'ExtraData_agentId',
       '_header_eventContext_category',
       'MetaData_RequestContext_Accept-Encoding', 'ExtraData_botResponse',
       '_header_eventContext_sourceType', 'ExtraData_inputMessage',
       'ExtraData_helixTime', '_header_eventContext_region',
       '_header_eventContext_ingestionTimestamp', 'ExtraData_intentTime',
       '_header_eventContext_endpoint', 'ExtraData_promptTranscript',
       '_header_source', 'MetaData_RequestContext_X-Forwarded-For',
       'ExtraData_ozmoTime', 'ExtraData_Time', 'MetaData_ServerTime',
       'MetaData_Client', 'MetaData_RequestContext_X-Fo

,ExtraData_inputMessage
Wrong issue,14
That worked!,11
Still need help,6


In [4]:
home_df = run_query(
"""
SELECT
element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') AutoFeedback,
element_at(VHE.edp_raw_data_map, 'Name')
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    --AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND element_at(VHE.edp_raw_data_map, 'Scope')='GenerativeAISearchBotVoice'
    AND (element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') in ('That worked!', 'Still need help', 'Wrong issue') or element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') is null)
    AND (CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) in (573190, 573276, 572909, 573192, 573585, 573573, 572815, 552121, 576911, 572247)
    OR CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) is null)
    AND DATE(edp_updated_date) > DATE('2023-11-10')
    
""")
home_df

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_17932\494237659.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,AutoFeedback,_col1
0,None,RequestSummaryVoice
1,Wrong issue,RequestSummaryVoice
2,None,RequestSummaryVoice
3,None,RequestSummaryVoice
4,None,RequestSummaryVoice
...,...,...
831,None,RequestSummaryVoice
832,None,RequestSummaryVoice
833,None,RequestSummaryVoice
834,None,RequestSummaryVoice


In [5]:
df=run_query(file_to_string(r'G:\Decision_Support\Reporting\Business Insights\Power BI\Generative AI Dashboards\Expert Assist Pilot\AdHocs\8. Finding Proactive Messages\ExpertAssistResponsesByType.sql'))


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_17932\494237659.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


In [6]:
df[['Proactive Messages','Reactive Messages']].sum()

Proactive Messages     805
Reactive Messages     1959
dtype: int64